In [ ]:
import pandas as pd
from datetime import date, datetime, timedelta as td
# import numpy as np
import xml.etree.ElementTree as ET



In [ ]:
tree = ET.parse("file+path/export.xml")

attribute_list = []

for child in tree.getroot():
    child_attribute = child.attrib
    for i in list(child):
        metadata_values = list(i.attrib.values())
        if len(metadata_values) == 2:
            data_dict = {metadata_values[0]: metadata_values[1]}
            child_attribute.update(data_dict)

    attribute_list.append(child_attribute)

apple_health_df = pd.DataFrame(attribute_list)

# Remove long identifiers
apple_health_df.type = apple_health_df.type.str.replace('HKQuantityTypeIdentifier', "")
apple_health_df.type = apple_health_df.type.str.replace('HKCategoryTypeIdentifier', "")
apple_health_df.columns = apple_health_df.columns.str.replace("HKCharacteristicTypeIdentifier", "")

In [ ]:
display(apple_health_df)

In [ ]:
workout = spark.read.options(header='True').csv("/mnt/datalake/Bronze/apple_health_export/source/csv/Workout.csv")

display(workout.limit(10))

In [ ]:
#view column data types
workout.dtypes


In [ ]:
workout.createOrReplaceTempView("workout")

In [ ]:
%sql

select sum(totalEnergyBurned), date(startDate),  sum(duration), workoutActivityType from workout where date(startDate) = '2021-06-20' group by date(startDate), workoutActivityType order by startDate, workoutActivityType

In [ ]:
new_workout_df = workout.withColumnRenamed("startDate", "activityDate")
(new_workout_df
  .select("activityDate")
  .show(5, False))

In [ ]:
# # In Python
# final_workout_df = (workout
#   .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy"))
#   .drop("CallDate") 
#   .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy"))
#   .drop("WatchDate") 
#   .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), 
#   "MM/dd/yyyy hh:mm:ss a"))
#   .drop("AvailableDtTm"))

# # Select the converted columns
# (fire_ts_df
#   .select("IncidentDate", "OnWatchDate", "AvailableDtTS")
#   .show(5, False))